# Обрезка датасета

In [1]:
!gdown --id 1oF9cD-Fp4Un-xVsRdXfN7tX8A2m5iRGl


/opt/anaconda3/lib/python3.11/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1oF9cD-Fp4Un-xVsRdXfN7tX8A2m5iRGl
From (redirected): https://drive.google.com/uc?id=1oF9cD-Fp4Un-xVsRdXfN7tX8A2m5iRGl&confirm=t&uuid=e0a3909f-c9a6-4141-9359-a4dab241a06e
To: /Users/pavelkorkodinov/Hacks/BEST HACK RWB/central-fed-district-251113.osm.pbf
100%|████████████████████████████████████████| 844M/844M [00:38<00:00, 21.9MB/s]


In [2]:
!ls -lh


total 1771624
-rw-r--r--  1 pavelkorkodinov  staff    12K 15 ноя 16:04 RWB geo.ipynb
-rw-r--r--  1 pavelkorkodinov  staff   805M 15 ноя 11:49 central-fed-district-251113.osm.pbf
-rw-r--r--  1 pavelkorkodinov  staff    48M 15 ноя 15:42 moscow_small.pbf


In [9]:
# !pip install pyrosm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 4.0 MB/s eta 0:00:0000:0100:010m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached geopandas-1.1.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached shapely-2.1.2-cp311-cp311-macosx_10_9_x86_64.whl.metadata (6.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pyrobuf-0.9.3-cp311-cp311-macosx_10_9_x86_64.whl
  Using cached pyogrio-0.11.1.tar.gz (287 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [29 lines of output]
      Traceback

In [34]:
!osmium extract --bbox=37.5,55.5,37.8,55.9 -o moscow_small.osm.pbf "central-fed-district-251113.osm.pbf"


Open failed for 'moscow_small.osm.pbf': File exists. Try using --overwrite if you are sure you want to overwrite the file.


In [38]:
ls -lh


total 1870008
-rw-r--r--  1 pavelkorkodinov  staff    16K 15 ноя 16:08 RWB geo.ipynb
-rw-r--r--  1 pavelkorkodinov  staff   805M 15 ноя 11:49 central-fed-district-251113.osm.pbf
-rw-r--r--  1 pavelkorkodinov  staff    48M 15 ноя 16:08 moscow_small.osm.pbf
-rw-r--r--  1 pavelkorkodinov  staff    48M 15 ноя 16:07 moscow_small.pbf


# Базовое решение
На вход — текст адреса. На выход — JSON с несколькими кандидатами (locality, street, number, lon, lat, score). Базовый алгоритм = извлечение адресных объектов → простая нормализация/парсинг запроса → exact lookup по (street, housenumber) → fallback: все объекты по улице → fallback: fuzzy по названиям улиц → spatial fallback.

## Первичный взгляд на данные

In [55]:
from pyrosm import OSM
import json

osm = OSM("moscow_small.osm.pbf")

# Загружаем здания
buildings = osm.get_buildings()

# Берём первую запись
first_row = buildings.iloc[2]  # dropna чтобы убрать пустые столбцы

# Преобразуем в обычный словарь (строки, числа и т.д.)
record = first_row.to_dict()

# В GeoPandas столбец "geometry" содержит объект shapely → преобразуем в WKT
if "geometry" in record:
    record["geometry"] = record["geometry"].wkt

# Печатаем красиво отформатированный JSON
print(json.dumps(record, indent=2, ensure_ascii=False))


/opt/anaconda3/lib/python3.11/site-packages/geopandas/array.py:340: UserWarning: Cannot set the CRS, falling back to None. The CRS support requires the 'pyproj' package, but it is not installed or does not import correctly. The functions depending on CRS will raise an error or may produce unexpected results.
  self.crs = crs
/opt/anaconda3/lib/python3.11/site-packages/geopandas/geodataframe.py:465: UserWarning: Cannot set the CRS, falling back to None. The CRS support requires the 'pyproj' package, but it is not installed or does not import correctly. The functions depending on CRS will raise an error or may produce unexpected results.
  level.array.crs = crs
/opt/anaconda3/lib/python3.11/site-packages/geopandas/geodataframe.py:62: UserWarning: Cannot set the CRS, falling back to None. The CRS support requires the 'pyproj' package, but it is not installed or does not import correctly. The functions depending on CRS will raise an error or may produce unexpected results.
  data.crs = crs

{
  "addr:city": null,
  "addr:country": null,
  "addr:full": null,
  "addr:housenumber": "1",
  "addr:housename": null,
  "addr:postcode": null,
  "addr:place": null,
  "addr:street": "Осташковское шоссе",
  "email": null,
  "name": "Лемана про",
  "opening_hours": "Mo-Su 07:00-24:00",
  "operator": "Лемана Про",
  "phone": null,
  "ref": "5",
  "url": null,
  "visible": false,
  "website": null,
  "building": "retail",
  "amenity": null,
  "building:flats": null,
  "building:levels": null,
  "building:material": null,
  "building:min_level": null,
  "building:use": null,
  "craft": null,
  "height": null,
  "internet_access": null,
  "landuse": null,
  "levels": null,
  "office": null,
  "shop": "doityourself",
  "source": null,
  "start_date": null,
  "wikipedia": null,
  "id": 23169215,
  "timestamp": 1720601134,
  "version": 17,
  "geometry": "POLYGON ((37.667274475097656 55.899810791015625, 37.66726303100586 55.90010452270508, 37.667625427246094 55.90021896362305, 37.667984008789

In [42]:
from pyrosm import OSM
osm = OSM("moscow_small.osm.pbf")
buildings = osm.get_buildings()
buildings.head()


/opt/anaconda3/lib/python3.11/site-packages/geopandas/array.py:340: UserWarning: Cannot set the CRS, falling back to None. The CRS support requires the 'pyproj' package, but it is not installed or does not import correctly. The functions depending on CRS will raise an error or may produce unexpected results.
  self.crs = crs
/opt/anaconda3/lib/python3.11/site-packages/geopandas/geodataframe.py:465: UserWarning: Cannot set the CRS, falling back to None. The CRS support requires the 'pyproj' package, but it is not installed or does not import correctly. The functions depending on CRS will raise an error or may produce unexpected results.
  level.array.crs = crs
/opt/anaconda3/lib/python3.11/site-packages/geopandas/geodataframe.py:62: UserWarning: Cannot set the CRS, falling back to None. The CRS support requires the 'pyproj' package, but it is not installed or does not import correctly. The functions depending on CRS will raise an error or may produce unexpected results.
  data.crs = crs

,addr:city,addr:country,addr:full,addr:housenumber,addr:housename,addr:postcode,addr:place,addr:street,email,name,...,source,start_date,wikipedia,id,timestamp,version,tags,osm_type,geometry,changeset
0,Москва,None,None,22,None,None,None,улица Пришвина,None,"Торговый центр ""Александр Лэнд""",...,None,None,None,23056826,1698766639,62,"{""building:colour"":""white;yellow"",""name:uk"":""\...",way,"POLYGON ((37.60206 55.88612, 37.60224 55.88599...",NaN
1,None,None,None,119 с70,None,None,None,проспект Мира,None,Павильон №70 «Москва»,...,None,1967..1968,ru:Павильон № 70 «Москва» на ВДНХ,23161470,1732153372,15,"{""building:architecture"":""modernist"",""name:en""...",way,"POLYGON ((37.64055 55.82907, 37.64068 55.82922...",NaN
2,None,None,None,1,None,None,None,Осташковское шоссе,None,Лемана про,...,None,None,None,23169215,1720601134,17,"{""contact:instagram"":""https://www.instagram.co...",way,"POLYGON ((37.66727 55.89981, 37.66726 55.9001,...",NaN
3,Мытищи,RU,None,1,None,141031,None,Осташковское шоссе,None,Ашан,...,None,None,None,23169216,1694908891,41,"{""addr:district"":""\u0433\u043E\u0440\u043E\u04...",way,"POLYGON ((37.66381 55.89762, 37.66745 55.89877...",NaN
4,None,None,None,16 c1,None,None,None,Олимпийский проспект,None,None,...,None,None,None,23170575,1708938190,25,"{""construction"":""stadium"",""description"":""\u042...",way,"POLYGON ((37.62544 55.78178, 37.62565 55.78188...",NaN


In [62]:
buildings.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 131604 entries, 0 to 131603
Data columns (total 41 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   addr:city           21277 non-null   object  
 1   addr:country        12610 non-null   object  
 2   addr:full           141 non-null     object  
 3   addr:housenumber    82256 non-null   object  
 4   addr:housename      6 non-null       object  
 5   addr:postcode       13141 non-null   object  
 6   addr:place          3851 non-null    object  
 7   addr:street         78909 non-null   object  
 8   email               127 non-null     object  
 9   name                6509 non-null    object  
 10  opening_hours       1224 non-null    object  
 11  operator            935 non-null     object  
 12  phone               581 non-null     object  
 13  ref                 1412 non-null    object  
 14  url                 4 non-null       object  
 15  visible  

In [72]:
print(buildings.describe(include='all').T)

                       count  unique  \
addr:city              21277      65   
addr:country           12610       1   
addr:full                141     130   
addr:housenumber       82256   14565   
addr:housename             6       6   
addr:postcode          13141     428   
addr:place              3851     220   
addr:street            78909    2945   
email                    127     125   
name                    6509    5807   
opening_hours           1224     474   
operator                 935     330   
phone                    581     552   
ref                     1412    1190   
url                        4       4   
visible               121249       1   
website                  826     782   
building              131604     160   
amenity                 3472      89   
building:flats         17929     792   
building:levels        72643      83   
building:material      18061     116   
building:min_level       199       7   
building:use              99      12   


In [74]:
pip install pyrosm rapidfuzz rtree shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.8 kB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


## Шаг A — извлечь и сохранить адресные объекты

In [ ]:
# Шаг A: Извлечение адресов из moscow_small.osm.pbf
from pyrosm import OSM
import pandas as pd

# Путь к файлу
PBF_FILE = "moscow_small.osm.pbf"

# Загружаем данные
osm = OSM(PBF_FILE)

# Берём все объекты с тегом addr:housenumber
nodes = osm.get_data_by_custom_criteria(
    custom_filter={
        "addr:housenumber": True,
        "addr:street": True
    },
    filter_type="keep"
)

print("Columns in GeoDataFrame:", nodes.columns)

# Переименуем id в osm_id для единообразия
if 'id' in nodes.columns:
    nodes = nodes.rename(columns={'id':'osm_id'})

import ast  # безопасный способ превратить строку вида "{'key':'value'}" в словарь

def extract_tag(row, tag_name, default=None):
    tags = row['tags']
    if isinstance(tags, str):
        try:
            # превращаем строку в словарь
            tags = ast.literal_eval(tags)
        except:
            tags = {}
    if tags and tag_name in tags:
        return tags[tag_name]
    return default

# Теперь применяем
nodes['street'] = nodes.apply(lambda r: extract_tag(r, 'addr:street'), axis=1)
nodes['city'] = nodes.apply(lambda r: extract_tag(r, 'addr:city', default="Москва"), axis=1)
nodes['name'] = nodes.apply(lambda r: extract_tag(r, 'name'), axis=1)
nodes['building'] = nodes.apply(lambda r: extract_tag(r, 'building'), axis=1)

# Фильтруем по bbox Москвы (пример, можно подогнать)
min_lon, min_lat = 37.50, 55.55
max_lon, max_lat = 37.85, 55.90
nodes = nodes[nodes.geometry.apply(lambda g: g.centroid.x >= min_lon and g.centroid.x <= max_lon and
                                               g.centroid.y >= min_lat and g.centroid.y <= max_lat)]

# Выбираем только нужные колонки
columns = ['osm_id', 'city', 'addr:street', 'addr:housenumber', 'name', 'building', 'geometry', 'lon', 'lat']
existing_cols = [c for c in columns if c in nodes.columns]
nodes = nodes[existing_cols]

# Сохраняем результат для следующего шага
nodes.to_parquet("moscow_addresses.parquet", index=False)
print("Saved", len(nodes), "addresses")


/opt/anaconda3/lib/python3.11/site-packages/geopandas/array.py:340: UserWarning: Cannot set the CRS, falling back to None. The CRS support requires the 'pyproj' package, but it is not installed or does not import correctly. The functions depending on CRS will raise an error or may produce unexpected results.
  self.crs = crs
/opt/anaconda3/lib/python3.11/site-packages/geopandas/geodataframe.py:465: UserWarning: Cannot set the CRS, falling back to None. The CRS support requires the 'pyproj' package, but it is not installed or does not import correctly. The functions depending on CRS will raise an error or may produce unexpected results.
  level.array.crs = crs


In [ ]:
nodes['addr:street']

## Шаг B — нормализация и подготовка индексов
Сохраним преднормализованные поля в CSV/Parquet, чтобы во время запроса не нормализовать всё заново.

In [ ]:
# функция нормализации текста
ABBR = {
    "ул": "улица", "ул.": "улица",
    "пр": "проспект", "просп": "проспект", "пр-т": "проспект", "просп.": "проспект",
    "пер": "переулок", "ш": "шоссе", "бул": "бульвар",
    "наб": "набережная", "пл": "площадь", "д": "дом", "корп": "корпус", "стр": "строение"
}

# build_index.py
import pandas as pd
import re
from rtree import index
from rapidfuzz import fuzz

df = pd.read_parquet("moscow_addresses.parquet")

def normalize_text(s: str):
    if not s: return ""
    s = s.lower()
    s = re.sub(r'[^\w\s\-\/]', ' ', s)   # убрать пунктуацию, оставить /- (для 10/1, 12-14)
    s = re.sub(r'\s+', ' ', s).strip()
    # replace abbrev
    for k,v in ABBR.items():
        s = re.sub(r'\b' + re.escape(k) + r'\b', v, s)
    return s

df['street_norm'] = df['addr:street'].fillna('').apply(normalize_text)
df['housenorm'] = df['addr:housenumber'].fillna('').str.lower().str.strip()

print(df.columns)

# save normalized table
df.to_parquet("moscow_addresses_norm.parquet", index=False)

# build in-memory dicts and R-tree
from collections import defaultdict
dict_exact = defaultdict(list)
dict_street = defaultdict(list)

coords = df[['lon','lat']].values.tolist()
for i, row in df.iterrows():
    s = row['street_norm']
    h = row['housenorm']
    obj = row.to_dict()
    if s and h:
        dict_exact[(s,h)].append(obj)
    if s:
        dict_street[s].append(obj)
# build rtree for spatial fallback
rt = index.Index()
for i, row in df.iterrows():
    lon = row.get('lon')
    lat = row.get('lat')
    if lon is None or lat is None or pd.isna(lon) or pd.isna(lat):
        continue  # пропускаем записи без координат
    rt.insert(i, (lon, lat, lon, lat))

# optionally pickle dicts or keep df+rtree on process memory


## Шаг C — парсер входной строки (простейший)

Функция выделяет номер дома (первую «числовую» часть) и оставшееся считает улицей.

In [ ]:
import re
def extract_house_and_street(query):
    q = query.lower()
    q = re.sub(r'[.,;]', ' ', q)
    # normalize abbreviations here too
    for k,v in ABBR.items():
        q = re.sub(r'\b'+re.escape(k)+r'\b', v, q)
    # find housenumber pattern (examples: 10, 10/1, 12a, 12 к1, 12к1)
    m = re.search(r'(\d+[\/\-\dA-Za-zА-Яа-я]*)', q)
    if m:
        housen = m.group(1).replace(' ', '')
        street = (q[:m.start()] + q[m.end():]).strip()
        return housen, normalize_text(street)
    else:
        return None, normalize_text(q)


## Шаг D — функция поиска (базовая логика)

Псевдокод с финальным JSON-выходом:

In [ ]:
from rapidfuzz import fuzz

def geocode(query, topk=5):
    house, street_guess = extract_house_and_street(query)
    results = []

    # exact match
    if house:
        key = (street_guess, house)
        if key in dict_exact:
            for o in dict_exact[key]:
                # tiny variation score (например по координатам или id)
                score = 1.0 - 0.001*o['osm_id']%0.05
                results.append((o, score))

    # street exact (no house)
    if not results and street_guess in dict_street:
        for idx,o in enumerate(dict_street[street_guess][:topk]):
            score = 0.7 - 0.01*idx  # слегка различаем top-k
            results.append((o, score))

    # fuzzy street
    if not results:
        scores = []
        for s in dict_street.keys():
            r = fuzz.token_sort_ratio(s, street_guess)
            if r >= 70:
                scores.append((r, s))
        scores.sort(reverse=True)
        for r, s in scores[:10]:
            for idx,o in enumerate(dict_street[s][:topk]):
                score = 0.6*(r/100) - 0.01*idx
                results.append((o, score))

    # spatial fallback
    if not results:
        center = (37.6173, 55.7558)
        nearest_ids = list(rt.nearest((center[0],center[1],center[0],center[1]), 10))
        for idx,i in enumerate(nearest_ids):
            row = df.iloc[i].to_dict()
            score = 0.3 - 0.01*idx
            results.append((row, score))

    # сортировка и top-k
    results_sorted = sorted(results, key=lambda x: -x[1])[:topk]
    objs = []
    for o, score in results_sorted:
        objs.append({
            "locality": o.get('city') or 'г. Москва',
            "street": o.get('addr:street'),
            "number": o.get('addr:housenumber'),
            "lon": o.get('lon'),
            "lat": o.get('lat'),
            "score": round(score,3)
        })
    return {"searched_address": query, "objects": objs}


# Шаг E — тесты и базовая валидация

In [ ]:
# берем первые 3 адреса
test_addresses = []
for _, row in df.head(3).iterrows():
    street = row.get('addr:street') or ''
    housenumber = row.get('addr:housenumber') or ''
    city = row.get('addr:city') or 'Москва'
    query = f"{city}, {street} {housenumber}".strip()
    test_addresses.append(query)

# тестируем геокодер
import json

for addr in test_addresses:
    print("=== Исходный адрес ===")
    print(addr)
    result = geocode(addr)
    print("=== Результат геокодера ===")
    print(json.dumps(result, indent=2, ensure_ascii=False))
    print("\n")


In [ ]:
df

In [46]:
!pip install dataframe_image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 kB 3.5 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 6.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.0/497.0 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.7/385.7 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 23.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 7.2 MB/s eta 0:00:00
  Created wheel for greenlet: filename=greenlet-3.2.4-cp311-cp311-macosx_10_9_x86_64.whl size=222319 sha256=cbe0d8490cfe403b7a01608a82ff7ac53ebaf92fae5c4dc58f9fab0bdd32e879
  Stored in directo

In [ ]:
from pyrosm import OSM

osm = OSM("central-fed-district-251113.osm.pbf")
place = osm.get_data_by_custom_criteria(custom_filter={"building": True}, filter_type="keep")
place.head()


In [ ]:
osm.list_layers()


In [ ]:
buildings = osm.get_buildings()
len(buildings), buildings.head()
